In [103]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import Chrome
from selenium import webdriver
import re
import pandas as pd
import datetime
import uuid
from datetime import date
from selenium.webdriver.chrome.options import Options

In [74]:
main_url = 'http://cronica.diputados.gob.mx/DDebates/index.html'

In [12]:
options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/cblanesg/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


In [75]:
driver.get(main_url)

In [97]:
def month_to_number(m):
    if m == 'ene':
        return(1)
    elif m == 'feb':
        return(2)
    elif m == 'mar':
        return(3)
    elif m == 'abr':
        return(4)
    elif m == 'may':
        return(5)
    elif m == 'jun':
        return(6)
    elif m == 'jul':
        return(7)
    elif m == 'ago':
        return(8)
    elif m == 'sep':
        return(9)
    elif m == 'oct':
        return(10)
    elif m == 'nov':
        return(11)
    elif m == 'dic':
        return(12)

def get_date_speech(year_session, link_speech, text_day):
    day_speech = re.findall('[0-9]+', text_day)[0]
    month_speech = month_to_number(link_speech.split('/')[-2:][0])
    return(datetime.date(int(year_session), int(month_speech), int(day_speech)))

def get_links_legislaturas(legislaturas):
    """
    Parametros:
    -----------
    legislatura: list of int numbers of legislatura 
    
    Returns:
    --------
    lista de diccionario con los a el htmls de los debates legislativos
    """
    all_data = []
    
    for legislatura in legislaturas:
        link_debates = 'http://cronica.diputados.gob.mx/DDebates/' + str(legislatura) + '/index.html'
        
        driver.get(link_debates)
        
        links_sesiones = []
        type_sesion = []
        for h in driver.find_elements_by_css_selector('a'):
            links_sesiones.append(h.get_attribute('href'))
            type_sesion.append(re.sub('^[ ]+', '', h.text))
        
        
        for x, y in zip(links_sesiones, type_sesion):
            driver.get(x)
            
            for i in driver.find_elements_by_css_selector('a'):
                year_session = re.findall('[0-9]{4}', driver.find_element_by_css_selector('body > table:nth-child(3) > tbody > tr > td').text)[0]
                link_ = i.get_attribute('href')
                
                fecha_speech = get_date_speech(year_session, link_, i.text)
        
                temp = {}
                temp['id_debate'] = str(uuid.uuid3(uuid.NAMESPACE_URL,link_))
                temp['legislatura'] = legislatura
                temp['date_debate'] = fecha_speech
                temp['type_sesion'] = y
                temp['link_debate'] = link_
                all_data.append(temp)
    return(all_data)
    

In [105]:
htmls_speeches = get_links_legislaturas([63, 64])

In [106]:
pd.DataFrame(htmls_speeches)

,id_debate,legislatura,date_debate,type_sesion,link_debate
0,c0ceba86-4f50-3a1f-8907-e83005ee8720,63,2015-08-29,Primer Periodo Ordinario,http://cronica.diputados.gob.mx/DDebates/63/1e...
1,fb1799c9-00e9-3a85-b870-a54dbd24512a,63,2015-09-01,Primer Periodo Ordinario,http://cronica.diputados.gob.mx/DDebates/63/1e...
2,71979612-519d-32bd-ba03-5ce6c7b92e18,63,2015-09-03,Primer Periodo Ordinario,http://cronica.diputados.gob.mx/DDebates/63/1e...
3,37ea8705-c8f0-306d-91cf-6801203235a8,63,2015-09-08,Primer Periodo Ordinario,http://cronica.diputados.gob.mx/DDebates/63/1e...
4,a530b0c6-94a2-3b22-955f-5cb57544dea5,63,2015-09-10,Primer Periodo Ordinario,http://cronica.diputados.gob.mx/DDebates/63/1e...
...,...,...,...,...,...
373,b3f2f4ad-b4cb-3ef7-8c27-304f1f4565c4,64,2020-03-19,Segundo Periodo Ordinario,http://cronica.diputados.gob.mx/DDebates/64/2d...
374,7f1c9c76-f4aa-3772-a37c-95d627b805d4,64,2020-03-19,Segundo Periodo Ordinario,http://cronica.diputados.gob.mx/DDebates/64/2d...
375,a36388e6-7e70-3379-afbd-7560939c6952,64,2020-06-30,Primer Periodo Extraordinario,http://cronica.diputados.gob.mx/DDebates/64/2d...
376,d570d6a4-9083-3293-8d59-49f1939659dd,64,2020-07-22,Segundo Periodo Extraordinario,http://cronica.diputados.gob.mx/DDebates/64/2d...
